# keras CNN predict rectangle box

In [1]:
# !pip3 install tensorflow

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-07-24 14:56:52.722793: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 14:56:52.769305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 14:56:53.487497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = pickle_load_or_create('../models/rectangles_CNN_model_history', lambda: None, True)
print(model)

None


In [4]:
from DataGeneratorFrames import DataGeneratorRectangles

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
config = pickle_load_or_create('CNN_rectangles_96pixels', lambda:{
    'convolution': (3,3),
    'dim':256,
    'rgb':True,
    'unique_labels': 3,
}, config=True)
config

{'convolution': (3, 3), 'dim': 256, 'rgb': True, 'unique_labels': 3}

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, TimeDistributed, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

if model is None:
    model = Sequential()
    model.add(Conv2D(filters=24, kernel_size=config['convolution'],
                     input_shape=(config['dim'], config['dim'], 3 if config['rgb'] else 1)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=48, kernel_size=(3, 3)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Flatten())  # Flatten each frame
    model.add(Dense(64, activation='relu'))
    model.add(Dense(config['unique_labels'], activation='linear'))
    
    model.compile(optimizer=Adam(), 
                  loss='mean_absolute_error', 
                  metrics=['mean_absolute_error', 'mean_squared_error'])
else:
    model = model.model

/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 24)   │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 24)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 127, 127, 24)   │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 32)   │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 48)     │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 30, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,764,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,786,963 (10.63 MB)

 Trainable params: 2,786,755 (10.63 MB)

 Non-trainable params: 208 (832.00 B)

In [8]:
# Parameters
params = {'dim': (config['dim'],config['dim']),
          'n_classes': config['unique_labels'],
          'n_channels': 3 if config['rgb'] else 1,
          'shuffle': True,
}

training_generator = DataGeneratorRectangles(train=True, **params)
test_generator = DataGeneratorRectangles(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called


In [9]:
training_generator.batch_order

,videoID,batch_nr_video,frame_amount,batch_id
0,7,17,32,0
1,4,62,32,1
2,5,50,32,2
3,3,5,32,3
4,7,34,32,4
...,...,...,...,...
310,2,4,32,310
311,7,33,32,311
312,7,21,32,312
313,4,42,32,313


In [10]:
%%time
X, y = training_generator.__getitem__(25)

 __getitem__(25)
CPU times: user 1.45 s, sys: 47.4 ms, total: 1.5 s
Wall time: 460 ms


In [11]:
X.shape

(32, 256, 256, 3)

In [12]:
y.shape

(32, 3)

In [13]:
y[:5]

array([[0.5, 0.5, 1. ],
       [0.5, 0.5, 1. ],
       [0.5, 0.5, 1. ],
       [0.5, 0.5, 1. ],
       [0.5, 0.5, 1. ]])

In [1]:
%%time
history = model.fit(training_generator, epochs=1, 
                    validation_data=test_generator, shuffle=False)

NameError: name 'model' is not defined

In [ ]:
training_generator.batch_order

In [ ]:
training_generator.__getitem__(4)

In [ ]:
pd.DataFrame(history.history)

In [ ]:
model.fit?

In [ ]:
with open('../models/frames_skillborder_CNN_model_96pixels_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)

In [ ]:
X, y = test_generator.__getitem__(14)

In [ ]:
model.predict(X), y